# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# import libraries
from sqlalchemy import create_engine
import nltk
nltk.download(['punkt', 'wordnet'])

import numpy as np
import re
import pandas as pd
import sys
import os
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# load data from database
engine = create_engine('sqlite:///ETL_Pipeline.db')
df = pd.read_sql_table('ETL_message05', engine)
X = df['message']
Y = df.drop(['id','message','original','genre'],axis = 1)

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    # Making all the text as lowercase
    text = text.lower()
    # Removing the punctuations using re module
    text = re.sub(r'[^\w\s]','', text)
    # Removing numbers
    text = re.sub(r'[0-9]+','', text)
    # Tokenizing the words
    tokens = word_tokenize(text)
    # Removing Stop words
    stop_words = set(stopwords.words('english'))
    tokens = [i for i in tokens if not i in stop_words]
    # Lemmatizing the tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    return tokens 

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('cnt_vect', CountVectorizer(tokenizer = tokenize)),
    ('tf_idf', TfidfTransformer()),
    ('mo_clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,random_state = 1)
pipeline.fit(X_train,Y_train)

Pipeline(memory=None,
     steps=[('cnt_vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        s...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
# Testing model for train set

train_pred = pipeline.predict(X_train)

for col in range(len(Y.columns)):
    print(Y.columns[col],":")
    print(classification_report(Y_train.iloc[:,col], train_pred[:,col]))
#     print(classification_report(Y_train.iloc[:,col], train_pred.iloc[:,col], target_names = [Y.columns[col]]))

related :
             precision    recall  f1-score   support

          0       0.98      0.97      0.98      4572
          1       0.99      1.00      0.99     14955
          2       1.00      0.92      0.96       135

avg / total       0.99      0.99      0.99     19662

request :
             precision    recall  f1-score   support

          0       0.98      1.00      0.99     16327
          1       1.00      0.92      0.96      3335

avg / total       0.99      0.99      0.99     19662

offer :
             precision    recall  f1-score   support

          0       1.00      1.00      1.00     19570
          1       1.00      0.68      0.81        92

avg / total       1.00      1.00      1.00     19662

aid_related :
             precision    recall  f1-score   support

          0       0.98      1.00      0.99     11541
          1       0.99      0.97      0.98      8121

avg / total       0.98      0.98      0.98     19662

medical_help :
             precision    reca

In [9]:
# Testing model for test set

test_pred = pipeline.predict(X_test)

for col in range(len(Y.columns)):
    print(Y.columns[col],":")
    print(classification_report(Y_test.iloc[:,col], test_pred[:,col]))

related :
             precision    recall  f1-score   support

          0       0.62      0.46      0.53      1550
          1       0.84      0.91      0.87      4951
          2       0.71      0.09      0.17        53

avg / total       0.79      0.80      0.79      6554

request :
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5415
          1       0.80      0.44      0.57      1139

avg / total       0.88      0.88      0.87      6554

offer :
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6528
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6554

aid_related :
             precision    recall  f1-score   support

          0       0.74      0.86      0.80      3815
          1       0.75      0.58      0.65      2739

avg / total       0.74      0.74      0.74      6554

medical_help :
             precision    reca

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
parameters = {
#     'cnt_vect__max_features' : [None, 10000, 50000],
#     'tf_idf__use_idf' : [True, False],
    'mo_clf__estimator__n_estimators' : [100,200,300], 
    'mo_clf__estimator__min_samples_split' : [2,3,4]
}

cv = GridSearchCV(pipeline, param_grid = parameters, verbose = 2, n_jobs = -1)

In [11]:
# parameters = {
# #     'cnt_vect__max_features' : [None, 10000, 50000],
# #     'tf_idf__use_idf' : [True, False],
#     'mo_clf__estimator__n_estimators' : [100,200,300], 
#     'mo_clf__estimator__min_samples_split' : [2,3,4]
# }

# cv = GridSearchCV(pipeline, param_grid = parameters, verbose = 1)

In [ ]:
# Get best parameters
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] mo_clf__estimator__min_samples_split=2, mo_clf__estimator__n_estimators=100 
[CV]  mo_clf__estimator__min_samples_split=2, mo_clf__estimator__n_estimators=100, total= 5.9min
[CV] mo_clf__estimator__min_samples_split=2, mo_clf__estimator__n_estimators=100 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  6.6min remaining:    0.0s


[CV]  mo_clf__estimator__min_samples_split=2, mo_clf__estimator__n_estimators=100, total= 6.0min
[CV] mo_clf__estimator__min_samples_split=2, mo_clf__estimator__n_estimators=100 
[CV]  mo_clf__estimator__min_samples_split=2, mo_clf__estimator__n_estimators=100, total= 5.9min
[CV] mo_clf__estimator__min_samples_split=2, mo_clf__estimator__n_estimators=200 
[CV]  mo_clf__estimator__min_samples_split=2, mo_clf__estimator__n_estimators=200, total=11.7min
[CV] mo_clf__estimator__min_samples_split=2, mo_clf__estimator__n_estimators=200 


In [ ]:
cv.best_params_

In [ ]:
best_fit_model = cv.best_estimator_
print(best_fit_model)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# Testing model for train set using tuned model

train_pred = best_fit_model.predict(X_train)

for col in range(len(Y.columns)):
    print(Y.columns[col],":")
    print(classification_report(Y_train.iloc[:,col], train_pred[:,col]))
#     print(classification_report(Y_train.iloc[:,col], train_pred.iloc[:,col], target_names = [Y.columns[col]]))

In [ ]:
# Testing model for test set using tuned model

test_pred = best_fit_model.predict(X_test)

for col in range(len(Y.columns)):
    print(Y.columns[col],":")
    print(classification_report(Y_test.iloc[:,col], test_pred[:,col]))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
# pickle.dump(best_fit_model, open('Class.pkl', 'wb'))

In [ ]:
import pickle
import os.path

file_path = "classifier.pkl"
n_bytes = 2**31
max_bytes = 2**31 - 1
data = bytearray(n_bytes)

## write
bytes_out = pickle.dumps(best_fit_model)
with open(file_path, 'wb') as f_out:
    for idx in range(0, len(bytes_out), max_bytes):
        f_out.write(bytes_out[idx:idx+max_bytes])

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.